In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
google_api_key = os.getenv("GEMINI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

Google API Key exists and begins AI
Groq API Key exists and begins gsk_
OpenRouter API Key exists and begins sk-or-


In [3]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)

In [4]:
models = {"gemini": "gemini-2.5-pro", "groq": "openai/gpt-oss-120b"}

clients = {"gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq}

In [5]:
import platform
import shutil
import json
import subprocess


def retrieve_system_info():
    return {
        "platform": platform.platform(),
        "python_version": platform.python_version(),
        "machine": platform.machine(),
        "processor": platform.processor(),
    }


def rust_toolchain_info():
    rustc_path = shutil.which("rustc")
    cargo_path = shutil.which("cargo")
    rustc_ver = (
        subprocess.check_output(["rustc", "--version"], text=True).strip()
        if rustc_path
        else "rustc not found"
    )
    cargo_ver = (
        subprocess.check_output(["cargo", "--version"], text=True).strip()
        if cargo_path
        else "cargo not found"
    )
    return {
        "rustc": rustc_ver,
        "cargo": cargo_ver,
    }


system_info = retrieve_system_info()
rust_info = rust_toolchain_info()


system_info, rust_info

({'platform': 'Linux-6.17.0-8-generic-x86_64-with-glibc2.42',
  'python_version': '3.13.7',
  'machine': 'x86_64',
  'processor': ''},
 {'rustc': 'rustc not found', 'cargo': 'cargo not found'})

In [6]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Rust compiler to compile a single rust file called main.rs and then execute it in the simplest way possible.
Please reply with whether I need to install a Rust toolchain to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Rust code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.
Have the maximum possible runtime performance in mind; compile time can be slow. Fastest possible runtime performance for this platform is key.
Reply with the commands in markdown.

System information:
{system_info}

Rust toolchain information:
{rust_info}
"""

response = groq.chat.completions.create(
    model=models["groq"], messages=[{"role": "user", "content": message}]
)
display(Markdown(response.choices[0].message.content))

**Short answer:** Yes – you need to install a Rust tool‑chain.  
Below are the **minimal, one‑line steps** to get a fully‑functional `rustc` (and `cargo`) on your Linux machine, followed by the exact `compile_command` and `run_command` you can paste into Python.

---

## 1️⃣ Install the Rust tool‑chain (rustup)

```bash
# 1️⃣ Download and run the official installer (non‑interactive)
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

# 2️⃣ Add Rust to the current shell (or restart the terminal later)
source "$HOME/.cargo/env"
```

*What this does*  

- Installs **rustup** (the official Rust version manager).  
- Pulls the latest stable tool‑chain (`rustc`, `cargo`, `rust-std`, …).  
- Puts `~/.cargo/bin` on your `PATH` so `rustc` and `cargo` are instantly usable.

> **Verify** it worked:  
> ```bash
> rustc --version   # e.g. rustc 1.78.0 (9c6c5b7c1 2024-07-23)
> cargo --version
> ```

---

## 2️⃣ Compile for **maximum runtime performance**

When you care more about the speed of the produced binary than about compile time, use the most aggressive optimisation flags:

```bash
rustc -C opt-level=3 \
      -C target-cpu=native \
      -C lto=yes \
      -C codegen-units=1 \
      -C panic=abort \
      -C strip=symbols \
      main.rs -o main
```

Explanation of the flags  

| Flag | Why it helps runtime |
|------|----------------------|
| `-C opt-level=3` | Highest optimisation level. |
| `-C target-cpu=native` | Generates code tuned for *your* CPU (e.g., AVX, SSE). |
| `-C lto=yes` | Link‑time optimisation – whole‑program analysis. |
| `-C codegen-units=1` | Forces a single code‑gen unit → better inlining & optimisation. |
| `-C panic=abort` | Removes unwind code; panics abort immediately (smaller & faster). |
| `-C strip=symbols` | Removes debug symbols, reducing binary size (no effect on speed). |

---

## 3️⃣ Run the binary

```bash
./main
```

---

## 4️⃣ Put it together in Python

```python
import subprocess
from pathlib import Path

# -------------------------------------------------
# 1️⃣ Compile (fastest possible runtime performance)
# -------------------------------------------------
compile_command = [
    "rustc",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "lto=yes",
    "-C", "codegen-units=1",
    "-C", "panic=abort",
    "-C", "strip=symbols",
    "main.rs",          # source file
    "-o", "main"        # output executable
]

compile_result = subprocess.run(
    compile_command,
    check=True,
    text=True,
    capture_output=True
)
print("Compilation stdout:", compile_result.stdout)
print("Compilation stderr:", compile_result.stderr)

# -------------------------------------------------
# 2️⃣ Execute the compiled program
# -------------------------------------------------
run_command = ["./main"]          # the binary we just built
run_result = subprocess.run(
    run_command,
    check=True,
    text=True,
    capture_output=True
)

print("Program output:", run_result.stdout)
```

*What the Python code does*  

1. **Compiles** `main.rs` with the aggressive optimisation flags above.  
2. **Runs** the resulting executable (`./main`).  
3. Captures and returns the program’s `stdout`.

> **Tip:** If you ever need a quick “just‑works” compile (e.g., for debugging), you can drop all `-C …` flags and just run `rustc main.rs -O`. The flags shown are the *maximum‑performance* configuration.

---

## 5️⃣ (Optional) Verify the binary is truly native‑optimised

```bash
# Show the CPU‑specific instructions used (e.g., AVX2, SSE4.2, …)
objdump -d -M intel main | grep -E "avx|sse"
```

If you see AVX/SSE instructions, the `target-cpu=native` flag worked.

---

### 🎉 You’re now ready

* Rust is installed.  
* You can compile a single `main.rs` file into the fastest possible executable for your machine.  
* The exact `compile_command` and `run_command` for your Python workflow are provided above.

Happy coding! 🚀

In [7]:
compile_command = [
    "/Users/ed/.cargo/bin/rustc",
    "main.rs",
    "-C",
    "opt-level=3",
    "-C",
    "target-cpu=native",
    "-C",
    "codegen-units=1",
    "-C",
    "lto=fat",
    "-C",
    "panic=abort",
    "-C",
    "strip=symbols",
    "-o",
    "main",
]

run_command = ["./main"]

In [8]:
language = "Rust"  # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""


def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)},
    ]

In [10]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [11]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if "gpt" in model else None
    response = client.chat.completions.create(
        model=model, messages=messages_for(python), reasoning_effort=reasoning_effort
    )
    reply = response.choices[0].message.content
    reply = reply.replace("```cpp", "").replace("```rust", "").replace("```", "")
    return reply

In [12]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [13]:
python_hard = """
# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value


def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)

    random_numbers = [
        next(lcg_gen) % (max_val - min_val + 1) + min_val
        for _ in range(n)
    ]

    max_sum = float("-inf")

    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum

    return max_sum


def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)

    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)

    return total_sum


# Parameters
n = 10000          # Number of random numbers
initial_seed = 42  # Initial seed for the LCG
min_val = -10      # Minimum value of random numbers
max_val = 10       # Maximum value of random numbers


# Timing the function
import time

start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))

"""

In [14]:
def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(
            run_command, check=True, text=True, capture_output=True
        )
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [15]:
CSS = """
:root {
  --py-color: #209dd7;
  --cpp-color: #ecad0a;
  --accent:   #753991;
  --card:     #161a22;
  --text:     #e9eef5;
}

/* Full-width layout */
.gradio-container {
  max-width: 100% !important;
  padding: 0 40px !important;
}

/* Code card styling */
.card {
  background: var(--card);
  border: 1px solid rgba(255,255,255,.08);
  border-radius: 14px;
  padding: 10px;
}

/* Buttons */
.convert-btn button {
  background: var(--accent) !important;
  border-color: rgba(255,255,255,.12) !important;
  color: white !important;
  font-weight: 700;
}
.run-btn button {
  background: #202631 !important;
  color: var(--text) !important;
  border-color: rgba(255,255,255,.12) !important;
}
.run-btn.py button:hover  { box-shadow: 0 0 0 2px var(--py-color) inset; }
.run-btn.cpp button:hover { box-shadow: 0 0 0 2px var(--cpp-color) inset; }
.convert-btn button:hover { box-shadow: 0 0 0 2px var(--accent) inset; }

/* Outputs with color tint */
.py-out textarea {
  background: linear-gradient(180deg, rgba(32,157,215,.18), rgba(32,157,215,.10));
  border: 1px solid rgba(32,157,215,.35) !important;
  color: rgba(32,157,215,1) !important;
  font-weight: 600;
}
.cpp-out textarea {
  background: linear-gradient(180deg, rgba(236,173,10,.22), rgba(236,173,10,.12));
  border: 1px solid rgba(236,173,10,.45) !important;
  color: rgba(236,173,10,1) !important;
  font-weight: 600;
}

/* Align controls neatly */
.controls .wrap {
  gap: 10px;
  justify-content: center;
  align-items: center;
}
"""

In [16]:
with gr.Blocks(
    css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}"
) as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26,
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)", value="", language="cpp", lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model_options = list(clients.keys())
        model = gr.Dropdown(
            choices=model_options, value=model_options[0], show_label=False
        )
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(
                label="Python result", lines=8, elem_classes=["py-out"]
            )
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(
                label=f"{language} result", lines=8, elem_classes=["cpp-out"]
            )

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])


ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[2] Sandbox: CanCreateUserNamespace() clone() failure: EPERM
